In [1]:
path_to_titles = './../data/docs_latest_titles.tsv'

In [2]:
import pandas as pd
import numpy as np
import re
import os

In [3]:
!pip install pymorphy2
!pip install nltk

In [4]:
import pymorphy2
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
morph = pymorphy2.MorphAnalyzer()

from tqdm import notebook

nltk.download("stopwords")
# некоторые стоп слова за таковые не считаем, на всякий случай
# иначе что отвечать на запрос "ЧТО ГДЕ КОГДА"?
my_stopwords = list(set(stopwords.words('russian')) - set(['как', 'когда', 'почему', 'зачем', 'чтобы', 'что']))
my_stopwords[:5]

[nltk_data] Downloading package stopwords to /home/rodion/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['здесь', 'над', 'вас', 'мне', 'при']

In [5]:
# Кеширование очень сильно ускоряет парсинг (примерно в два раза)
PYMORPHY_CACHE = {}
def lemmatizer(words):
    global PYMORPHY_CACHE
    for word in words:
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = morph.parse(word)[0].normal_form
        yield PYMORPHY_CACHE[word_hash]

In [19]:
titles_df = pd.read_csv(path_to_titles, header=None, sep='\t')
titles_df.head()

,DocumentId,Title
0,0,SMARTJS ACADEMY АРХИТЕКТУРА JS ПРИЛОЖЕНИЙ ХАРЬКОВ
1,1,ESSAY ON IMPORTANCE OF CLEANLINESS AT HOME SCH...
2,2,ДОСКА НА ДВУХ КОЛЕСАХ КАК НАЗЫВАЕТСЯ
3,3,MY CAREER VIEW TOPIC EBONY TEEN UPSKIRT
4,4,EMBASSY OF THE REPUBLIC OF KAZAKHSTAN TO THE U...


In [24]:
import tqdm
tqdm.tqdm.pandas()
def process_title(title):
    titlewords = str(title).lower().split()
    titlewords = [word for word in titlewords if word not in my_stopwords 
                   and word not in stopwords.words('english')]
    title = ' '.join(lemmatizer(titlewords))

    return title
titles_df['Title'] = titles_df['Title'].progress_apply(process_title)
titles_df

100%|██████████| 582167/582167 [09:37<00:00, 1008.03it/s]


,DocumentId,Title
0,0,smartjs academy архитектура js приложение харьков
1,1,essay importance cleanliness home school neigh...
2,2,доска два колесо как называться
3,3,career view topic ebony teen upskirt
4,4,embassy republic kazakhstan united arab emirat...
...,...,...
582162,582162,урок 2 как получать заказ рейтинг яндекс такси...
582163,582163,nan
582164,582164,0c5v5ey insurance archive state south african ...
582165,582165,488204_120826113120_d5evo


In [25]:
titles_df.to_csv('titles_corrected', index=None)